Desicion Tree!! -360DIGITMG

Decision Tree is a Supervised learning technique that can be used for both classification and Regression problems.
Decision Tree uses a flowchart like a tree structure to show the predictions that result from a series of feature-based splits. It starts with a root node and ends with a decision made by leaves.

Random Forest
A random forest is a machine learning technique that’s used to solve regression and classification problems. It utilizes ensemble learning, which is a technique that combines many classifiers to provide solutions to complex problems.

A random forest algorithm consists of many decision trees. The ‘forest’ generated by the random forest algorithm is trained through bagging or bootstrap aggregating.

3. Use decision trees & random forest algorithm to prepare a model on fraud data, treating those who have taxable_income <= 30000 as "Risky" and others are "Good".

Business objective: Predict whether it is Risky or good to 

In [1]:
import pandas as pd               # for Data Manipulation
import numpy as np                #for Mathematical calculations
# Importing company dataset using pandas
fraud = pd.read_csv(r"C:\Users\D\Desktop\New Assignments  Keys\Datasets\Fraud_check.csv")

In [2]:
fraud.columns

Index(['Undergrad', 'Marital.Status', 'Taxable.Income', 'City.Population',
       'Work.Experience', 'Urban'],
      dtype='object')

In [3]:
### EDA, Pre-processing
fraud.info()
fraud.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital.Status   600 non-null    object
 2   Taxable.Income   600 non-null    int64 
 3   City.Population  600 non-null    int64 
 4   Work.Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.2+ KB


,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [4]:
# Target Variable has to be derived from taxable_income where taxable_income <= 30000 as "Risky" and others are "Good".
# let us form a new variable using binning using numpy .where function
import numpy as np
fraud['Risk_level'] = np.where(fraud['Taxable.Income'] <= 30000, 'Risky', 'Good')


In [5]:
# we drop the taxable_income column since we dont need it now
fraud.drop('Taxable.Income', axis = 1, inplace = True)

In [6]:
fraud['Risk_level'].value_counts(normalize = True)
# our dataset is imbalanced - almost 80-20 ratio

Good     0.793333
Risky    0.206667
Name: Risk_level, dtype: float64

In [7]:
# Missing values
fraud.isnull().sum()
# there are no missing values in our dataset

Undergrad          0
Marital.Status     0
City.Population    0
Work.Experience    0
Urban              0
Risk_level         0
dtype: int64

In [8]:
# duplicate entries
fraud[fraud.duplicated(keep = False)]
# there are no repeated rows

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,Risk_level


In [9]:
fraud.describe(include = 'all')
# we see that all numerical columns have s.dev > 0, therefor all are relevant

,Undergrad,Marital.Status,City.Population,Work.Experience,Urban,Risk_level
count,600,600,600.000000,600.000000,600,600
unique,2,3,NaN,NaN,2,2
top,YES,Single,NaN,NaN,YES,Good
freq,312,217,NaN,NaN,302,476
mean,NaN,NaN,108747.368333,15.558333,NaN,NaN
std,NaN,NaN,49850.075134,8.842147,NaN,NaN
min,NaN,NaN,25779.000000,0.000000,NaN,NaN
25%,NaN,NaN,66966.750000,8.000000,NaN,NaN
50%,NaN,NaN,106493.500000,15.000000,NaN,NaN
75%,NaN,NaN,150114.250000,24.000000,NaN,NaN


In [10]:
# let us explore the categorical variables
fraud.Undergrad.value_counts(normalize = True)

YES    0.52
NO     0.48
Name: Undergrad, dtype: float64

In [11]:
fraud['Marital.Status'].value_counts(normalize = True)

Single      0.361667
Married     0.323333
Divorced    0.315000
Name: Marital.Status, dtype: float64

In [12]:
fraud.Urban.value_counts(normalize = True)

YES    0.503333
NO     0.496667
Name: Urban, dtype: float64

all these variables have nearly = distinction between their own categories therefore all are relevant

In [13]:
# forming dummy variables for the categorical columns
fraud = pd.get_dummies(fraud, columns = ["Undergrad", "Marital.Status",'Urban','Risk_level'], drop_first = True)

In [14]:
# Input and Output Split
predictors = fraud.loc[:, fraud.columns!="Risk_level_Risky"]
target = fraud["Risk_level_Risky"]

In [15]:
# Splitting data into training and testing data set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.25, random_state=0)

In [16]:
from sklearn.tree import DecisionTreeClassifier as DT
model = DT(criterion = 'entropy')
model.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [17]:
# Prediction on Test Data
preds = model.predict(x_test)
pd.crosstab(y_test, preds, rownames=['Actual'], colnames=['Predictions'])

Predictions,0,1
Actual,,
0,77,36
1,29,8


In [18]:
np.mean(preds == y_test) # Test Data Accuracy 
# accuracy on train dataset is 57% which is very low

0.5666666666666667

In [19]:
# Prediction on Train Data
preds = model.predict(x_train)
pd.crosstab(y_train, preds, rownames = ['Actual'], colnames = ['Predictions'])

Predictions,0,1
Actual,,
0,363,0
1,0,87


In [20]:
np.mean(preds == y_train) # Train Data Accuracy
# prediction on train dataset is 100%

1.0

In [21]:
# let us try random forest
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=500, n_jobs=1, random_state=42)
rf_clf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=500, n_jobs=1, random_state=42)

In [22]:
from sklearn.metrics import accuracy_score, confusion_matrix
confusion_matrix(y_test, rf_clf.predict(x_test))
accuracy_score(y_test, rf_clf.predict(x_test))
# 71% accuracy, therefor our accuracy increased by 14% with the help of random forest, which is vey bigh level of increment

0.7133333333333334

In [23]:
confusion_matrix(y_train, rf_clf.predict(x_train))


array([[363,   0],
       [  0,  87]], dtype=int64)

In [24]:
accuracy_score(y_train, rf_clf.predict(x_train))

1.0

## Hyperparameter Tuning
As the model is overfitting, we do some hyperparameter tuning to get the desired result

In [25]:
# Creating new model testing with new parameters
forest_new = RandomForestClassifier(n_estimators=100,max_depth=10,min_samples_split=20,criterion='gini')  # n_estimators is the number of decision trees
forest_new.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_split=20)

In [26]:
print('Train accuracy: {}'.format(forest_new.score(x_train, y_train)))
print('Test accuracy: {}'.format(forest_new.score(x_test, y_test)))

Train accuracy: 0.8111111111111111
Test accuracy: 0.7533333333333333
